# Object oriented error handling
- Error are defined by classes
- an error object is instantiated at error time
    - the class used tells you something about the problem
    - the error object may include specific information about the problem, like a file path that doesn't exist
- Java - compilier DEMANDS you handle all kinds of potential errors
- Python - hey whatever...
- but, if you get an error, and there's no handler, you CRASH
- [List of builtin errors]
(https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
- error types form a hierarchy(formed by using single inheritance)


# can explicitly 'raise'(or throw) an error 

In [1]:
def oops(n):
    if n == 3:
        raise ValueError("oops is unhappy!",
                         'bad value n=', 
                         3)
    return n+1

oops(1)

2

In [2]:
oops(3)

ValueError: ('oops is unhappy!', 'bad value n=', 3)

# catch errors with try/except

In [3]:
try:
    oops(3)
except Exception as e:
    print(e, e.args)

'now we are here' 

('oops is unhappy!', 'bad value n=', 3) ('oops is unhappy!', 'bad value n=', 3)


'now we are here'

# 'finally' clause is always executed, with or without error
- gives you a chance to do something before
you lose control

In [4]:
try:
    oops(1)
except Exception:
    print('got error')
finally:
    print('finally')
'now we are here'

finally


'now we are here'

In [5]:
try:
    oops(3)
except Exception:
    print('got error')
finally:
    print('finally')
'now we are here'

got error
finally


'now we are here'

# call stack frames display
- if no error handler is found, the top
level will display a 'stack trace', and your entire computation is terminated
- shows context of error
- very useful for debugging
- note: the most recent call frame is printed last

In [ ]:
def A(n):
    B(1)

def B(n):
    C(2)

def C(n):
    D(3)
    
def D(n):
    raise Exception("stack frames will be displayed")

A(10)

# can define a custom error class
- usually inherit from 'Exception'

# breaking out of nested loops                    

In [ ]:
# note use of 'pass'

class breakNested(Exception):
    pass

def bn(bval):
    try:
        for x in range(3):
            print('x',x)
            for y in range(3):
                print('y',y)
                if y == bval:
                    raise breakNested
    except breakNested:
        pass

In [ ]:
bn(9)

In [ ]:
bn(2)

In [ ]:
class missedSecret(Exception):
    def __init__(self, secret, guess):
        self.secret = secret
        self.guess = guess
    
def guessSecret(guess):
    secret = 1234
    if guess != secret:
        raise missedSecret(secret, guess)
    return 'guess is correct'
    
guessSecret(1234)

In [ ]:
guessSecret(34)

# can get error object and examine it
- info in error object might help decide how to handle the error

In [ ]:
try:
    guessSecret(12)
except missedSecret as ms:
    # ms will be bound to the error object
    print('secret={} guess={}'.format(ms.secret, ms.guess))

# when an error is raised, Python will search the call stack for an error handler
- Python checks the current stack frame for a handler, then checks each
older frame in turn
- if no handler is found, the error is printed by the top level(and your program dies)

In [ ]:
# no error handler in 'bar', so look in caller, 'foo',
# which does have one.

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    # division by zero error raised here
    b/a

foo()

In [ ]:
# 'bar' has a handler, 
# so error is caught there

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except Exception as e:
        print('caught in bar:', e)

foo()

In [ ]:
# both 'foo' and 'bar' have handlers, but
# neither is the right type, so the error
# is NOT caught

def foo():
    try:
        bar()
    except FileNotFoundError as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except ValueError as e:
        print('caught in bar', e)

foo()

# Complex error example
- you can ignore this if you wish

In [ ]:
def tc(b, f):
    if b:
        try:
            return tc2(f)
        # usually a good idea to catch
        # Exception at top level
        except Exception as e:
            print('tc: caught {}'.format(e))
    else:
        try:
            return tc2(f)
        except OSError as os:
            print('tc: caught {}'.format(os))  

def tc2(f):
    try:
        return tc3(f)
    # can catch any number of error types
    # in a single try
    except FileNotFoundError as fe:
        print('tc2: caught {}'.format(fe))
    except MemoryError as me:
        print('tc2: caught {}'.format(me))
    
def tc3(f):
    try:
        return f()
    except ArithmeticError as ae:
        print('tc3: caught {}'.format(ae))  
    
def noproblem():
    a = 5/4
    return a
    
def dbz2():
    try: 
        c = 1/0
        return c
    except ZeroDivisionError as z:
        print('dbz: caught {}'.format(z))
        
def dbz3():
    a = 1/0
    return a
    
def si():
    raise StopIteration
    
def fnf():
    # can put useful information about 
    # the error into the error object
    raise FileNotFoundError('/tmp/foo')

def me():
    raise MemoryError

In [ ]:
# should run fine, and not generate an error

tc(1, noproblem)

In [ ]:
#  caught the error it generated, inside dbz2

tc(1, dbz2)

In [ ]:
# no handler in dbz3
# caught by handler in tc3

tc(1, dbz3)

In [ ]:
# caught in 'tc' Exception handler

tc(1, si)

In [ ]:
# no handler on the call stack
# top level prints stack trace, 
# and your program crashes

tc(0, si)